# Irrigation time-series attributes extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the area equipped for irrigation (AEI) between 1900 and 2005 from the Historical Irrigation Dataset (HID).

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* rasterio
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* data/shapefiles/estreams_catchments.shp
* data/irrigation/AEI_EARTHSTAT_IR_{1900, 1910, 1920, 1930, 1940, 1960, 1970, 1980, 1985, 1990, 1995, 2000, 2005}.asc https://mygeohub.org/publications/8 (Last access: 05 December 2023)

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Siebert, S., Kummu, M., Porkka, M., Döll, P., Ramankutty, N., and Scanlon, B. R.: A global data set of the extent of irrigated land from 1900 to 2005, Hydrol. Earth Syst. Sci., 19, 1521–1545, https://doi.org/10.5194/hess-19-1521-2015, 2015.

## Licenses
* CC0 - Creative Commons: https://mygeohub.org/publications/8 (Last access: 06 December 2023)

## Observations

* HID provides the AEI in 8 different products. Here we decided to use the AEI_EARTHSTAT_IR_{} version of available, which was the version used in HydroAtlas (yet only for year 2005) and other studies. 

# Import modules

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm
import glob
import rasterio
from rasterio.mask import geometry_mask
from rasterio.warp import calculate_default_transform

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.


In [ ]:
# Non-editable variables:
PATH_OUTPUT = "results/timeseries/irrigation/"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [ ]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries.head()

In [ ]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

## AEI files

In [ ]:
filenames =['data/irrigation/AEI_EARTHSTAT_IR_1900.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1910.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1920.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1930.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1940.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1950.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1960.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1970.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1980.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1985.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1990.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_1995.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_2000.asc',
            'data/irrigation/AEI_EARTHSTAT_IR_2005.asc']

## Computation processes

In [ ]:
# Initialize an empty DataFrame to store the results
irrigation_attributes_df = pd.DataFrame()

prefix_values = ["1900", "1910", "1920", "1930", "1940",
                "1950", "1960", "1970", "1980", "1985", 
                 "1990", "1995", "2000", "2005"]

# Define the CRS for EPSG:4326 (WGS 84) (same as the boundaries shapefile)
crs = 'EPSG:4326'

i = 0

for filename in filenames:
    
    # Open the ASC file to read metadata
    with rasterio.open(filename) as src:
        # Reproject the data to WGS84
        transform, width, height = calculate_default_transform(
            src.crs, crs, src.width, src.height, *src.bounds)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Create an empty list to store the sum
        mean_values = []

        for idx, geom in tqdm.tqdm(catchment_boundaries.iterrows()):
            # Check if the geometry is empty or invalid
            if geom['geometry'] is None or geom['geometry'].is_empty or not geom['geometry'].is_valid:
                avg_value = np.nan
            else:
                # Create a mask for the geometry
                mask = geometry_mask([geom['geometry']], out_shape=(height, width), transform=transform, invert=True)

                # Read the values within the geometry from the raster
                data = src.read(1, masked=True)
                values = data[mask]

            # Calculate statistics only if there are valid values in the 'values' array
            if len(values) > 0:
                
                avg_value = np.sum(values)
            else:
                # Handle the case when there are no valid values (e.g., by setting them to NaN or a specific value)
                avg_value = np.nan
                
            # Append the mean value to the list
            mean_values.append(avg_value)

            
    # Create a DataFrame to store the results for this file
    col_name = prefix_values[i]
    data = {
        'basin_id': catchment_boundaries['basin_id'],
        col_name: mean_values,
    }
    results_df = pd.DataFrame(data)
    results_df.set_index("basin_id", inplace=True)
    #results_df = results_df.add_prefix(prefix_values[i])

    # Concatenate the results with the final DataFrame
    irrigation_attributes_df = pd.concat([irrigation_attributes_df, results_df], axis=1)
    i = i + 1
    
irrigation_attributes_df = irrigation_attributes_df.T*0.01

In [ ]:
# We set the index's name to date
irrigation_attributes_df.index.name = "date"
irrigation_attributes_df

In [ ]:
# Here we sort the columns:
irrigation_attributes_df = irrigation_attributes_df.sort_index(axis=1)
irrigation_attributes_df

In [ ]:
# Round the data to 3 decimals
irrigation_attributes_df = irrigation_attributes_df.astype(float).round(3)
irrigation_attributes_df

# Data export

In [ ]:
# Export the final dataset:
irrigation_attributes_df.to_csv(PATH_OUTPUT+"estreams_irrigation_yearly.csv")

# End